In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import random
from scipy.misc import toimage
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator
import PIL
import sys
import seaborn as sns
import sklearn.metrics
import pickle

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
torch.cuda.set_device(1)
x=torch.Tensor(5,3)
print(x.cuda())

tensor([[-6.4262e-37,  4.5611e-41, -6.4262e-37],
        [ 4.5611e-41,  3.7835e-44,  0.0000e+00],
        [ 8.8199e-21,  4.5611e-41, -3.1951e+27],
        [ 3.0795e-41,  2.8026e-44,  0.0000e+00],
        [ 2.8026e-44,  0.0000e+00, -7.5962e-37]], device='cuda:1')


In [5]:
print(torch.__version__)

1.1.0a0+828a6a3


In [6]:
data_aug = True 
early_stopping = True

In [7]:
data_aug_transforms = []

if data_aug == True:

    data_aug_transforms = [
        transforms.RandomCrop((32,32), padding=4),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomRotation(15),
#         transforms.RandomAffine(degrees=0, translate=(.3,.7)),
#         transforms.ColorJitter(
#                 brightness=float(0.1*np.random.rand(1)),
#                 contrast=float(0.1*np.random.rand(1)),
#                 saturation=float(0.1*np.random.rand(1)),
#                 hue=float(0.1*np.random.rand(1))),
#         
#         transforms.Resize((224,224)),
        transforms.ColorJitter(hue=.25, saturation=.25),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
        transforms.RandomGrayscale(p=0.1)
                    ]

In [8]:
norm_transform = transforms.Compose(data_aug_transforms+[transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                     ])
test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                     ])

In [9]:
# cifar_dataset = torchvision.datasets.CIFAR10(root='dataset/',
#                                            train=True,
#                                            transform=norm_transform,
#                                            download=False)

# test_dataset = torchvision.datasets.CIFAR10(root='dataset/',
#                                           train=False,
#                                           transform=test_transform)

In [10]:
cifar_dataset = torchvision.datasets.CIFAR10(root='../../dataset/',
                                           train=True,
                                           transform=norm_transform,
                                           download=False)

test_dataset = torchvision.datasets.CIFAR10(root='../../dataset/',
                                          train=False,
                                          transform=test_transform)

In [24]:
n = 7
np.random.seed(n)
torch.cuda.manual_seed_all(n)
torch.manual_seed(n)

In [25]:
def weights_init(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight.data, mean=0.0, std=1e-3)
        m.bias.data.fill_(0.0)
        
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight.data) 
        m.bias.data.fill_(0.0)

def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device: %s'%device)

Using device: cuda


In [27]:
input_size = 32 * 32 * 3
layer_config= [512, 256]
num_classes = 10
num_epochs = 30
batch_size = 200
learning_rate = 0.01 #1e-3
learning_rate_decay = 0.99
reg=0#0.001
fine_tune = True
pretrained= True

In [28]:
num_training= 49000
num_validation =1000
mask = list(range(num_training))
train_dataset = torch.utils.data.Subset(cifar_dataset, mask)
mask = list(range(num_training, num_training + num_validation))
val_dataset = torch.utils.data.Subset(cifar_dataset, mask)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [29]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [30]:
class VggModel(nn.Module):
    def __init__(self, n_class, fine_tune, pretrained=True):
        super(VggModel, self).__init__()
        
        #load the pretrained model
        vgg11_bn = models.vgg11_bn(pretrained=pretrained)
        self.model = nn.Sequential()
        self.model.features = vgg11_bn.features
        set_parameter_requires_grad(self.model, fine_tune)

        #adding 2 linear layers
        self.model.classifier = nn.Sequential(
                    nn.Linear(in_features=layer_config[0], out_features=layer_config[1], bias=True),
                    nn.BatchNorm1d(layer_config[1]),
                    nn.ReLU(),
                    nn.Linear(in_features=layer_config[1], out_features=n_class, bias=True)
                        )

    def forward(self, x):

        x = self.model.features(x)
        x = x.squeeze()
        out = self.model.classifier(x)

        return out

In [31]:
class AlexModel(nn.Module):
    def __init__(self, n_class, fine_tune, pretrained=True):
        super(AlexModel, self).__init__()
        
        #load the pretrained model
        alexNet = models.alexnet(pretrained=pretrained)
        self.model = nn.Sequential()
        self.model.features = alexNet.features
        set_parameter_requires_grad(self.model, fine_tune)

        #adding 2 linear layers
        self.model.classifier = nn.Sequential(
                    nn.Linear(in_features=layer_config[0], out_features=layer_config[1], bias=True),
                    nn.BatchNorm1d(layer_config[1]),
                    nn.ReLU(),
                    nn.Linear(in_features=layer_config[1], out_features=n_class, bias=True)
                        )

    def forward(self, x):

        x = self.model.features(x)
        x = x.squeeze()
        out = self.model.classifier(x)

        return out

In [48]:
class ResNetModel(nn.Module):
    def __init__(self, n_class, fine_tune, pretrained=True):
        super(ResNetModel, self).__init__()
        
        #load the pretrained model
        resnet = models.resnet50(pretrained=pretrained)
        # delete the last fc layer.
        modules = list(resnet.children())[:-1]      
        self.model = nn.Sequential() 
        self.model.features = nn.Sequential(*modules)
        set_parameter_requires_grad(self.model, fine_tune)

        #adding 2 linear layers
        self.model.classifier = nn.Sequential(
                    nn.Linear(in_features=layer_config[0], out_features=layer_config[1], bias=True),
                    nn.BatchNorm1d(layer_config[1]),
                    nn.ReLU(),
                    nn.Linear(in_features=layer_config[1], out_features=n_class, bias=True)
                        )

    def forward(self, x):

        x = self.model.features(x)
        x = x.squeeze()
        out = self.model.classifier(x)

        return out

In [40]:
class InceptionModel(nn.Module):
    def __init__(self, n_class, fine_tune, pretrained=True):
        super(InceptionModel, self).__init__()
        
        #load the pretrained model
        inceptionNet = models.inception_v3(pretrained=pretrained)
        self.model = nn.Sequential()
        self.model.features = inception_v3.features
        set_parameter_requires_grad(self.model, fine_tune)

        #adding 2 linear layers
        self.model.classifier = nn.Sequential(
                    nn.Linear(in_features=layer_config[0], out_features=layer_config[1], bias=True),
                    nn.BatchNorm1d(layer_config[1]),
                    nn.ReLU(),
                    nn.Linear(in_features=layer_config[1], out_features=n_class, bias=True)
                        )

    def forward(self, x):

        x = self.model.features(x)
        x = x.squeeze()
        out = self.model.classifier(x)

        return out

In [41]:
def PrintModelSize(model, disp=True):
    model_sz = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Number of trainable parameters = ",model_sz)
    return model_sz

In [42]:
def trainAndTest(modelClass, num_classes, fine_tune, pretrained, nameOfModel):
    
    # Initialize the model for this run
    model= modelClass(num_classes, fine_tune, pretrained)
    
    # Print the model we just instantiated
    print(model)

    print("Params to learn:")
    if fine_tune:
        params_to_update = []

        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)

    else:
        params_to_update = model.parameters()
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    model.to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params_to_update, lr=learning_rate, weight_decay=reg)

    # Train the model
    lr = learning_rate
    Loss = []                           #to save all the model losses
    valAcc = []
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            # Move tensors to the configured device
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

            Loss.append(loss)               #save the loss so we can get accuracies later
        # Code to update the lr
        lr *= learning_rate_decay
        update_lr(optimizer, lr)
        with torch.no_grad():
            correct = 0
            total = 0
            for images, labels in val_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            current_valAcc = 100 * correct / total
            valAcc.append(current_valAcc)
            if current_valAcc >= np.amax(valAcc):
                torch.save(model.state_dict(),'TransferLearning_models/model'+str(epoch+1)+'.ckpt')

            print('Validataion accuracy is: {} %'.format(100 * correct / total))

    last_model = model
    last_model.eval()
    
    best_id = np.argmax(valAcc)
    best_model = modelClass(num_classes, fine_tune, pretrained).to(device)
    best_model.load_state_dict(torch.load('TransferLearning_models/model'+str(best_id+1)+'.ckpt'))
    best_model.eval()
    
    print('Testing ........................................')
    # Test the model
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = best_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accBestNet = 100 * correct / total
        print('Accuracy of the best network on the {} test images: {} %'.format(total, 100 * correct / total))
        print("Best Epoch: ", best_id+1)

    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = last_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the last network on the {} test images: {} %'.format(total, 100 * correct / total))
        
    # Save the best model checkpoint
    torch.save(best_model.state_dict(), 'TransferLearning_models/'+nameOfModel+'.ckpt')


    plt.plot(Loss, label = "Loss")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.legend(loc="upper left")
    plt.show()

    plt.plot(valAcc, label = "Val Acc")
    plt.xlabel("Epochs")
    plt.ylabel("Validation Accuracy")
    plt.legend(loc="upper right")
    plt.show()
    
    return best_model, accBestNet

In [36]:
vgg, accVGG = trainAndTest(VggModel, num_classes, fine_tune, pretrained, 'vggModel')

VggModel(
  (model): Sequential(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace)
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU(inplace)
      (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (13)

KeyboardInterrupt: 

In [37]:
alex, accALEX = trainAndTest(AlexModel , num_classes, fine_tune, pretrained, 'alexModel')

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.torch/models/alexnet-owt-4df8aa71.pth
244418560it [00:06, 37992602.16it/s]


AlexModel(
  (model): Sequential(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (classifier): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine

RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small at /tmp/pip-req-build-l1dtn3mo/aten/src/THCUNN/generic/SpatialDilatedMaxPooling.cu:54

In [49]:
resnet, accRES = trainAndTest(ResNetModel , num_classes, fine_tune, pretrained, 'resnetModel')

ResNetModel(
  (model): Sequential(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (dow

RuntimeError: Given input size: (2048x1x1). Calculated output size: (2048x-5x-5). Output size is too small at /tmp/pip-req-build-l1dtn3mo/aten/src/THCUNN/generic/SpatialAveragePooling.cu:47

In [ ]:
inception, accINCEPTION = trainAndTest(InceptionModel , num_classes, fine_tune, pretrained, 'inceptionModel')

In [ ]:
def VisualizeFilter(model, superTitle):
    
    tensor = torch.ones(model.conv[0].weight.data.size())
    tensor = model.conv[0].weight.clone().detach().requires_grad_(False)
    tensor = tensor.cpu().data.numpy()
    
    t_max = np.amax(tensor)
    t_min = np.amin(tensor)

    tensor = (tensor - t_min) / (t_max - t_min) 

    num_cols = 16
    num_kernels = tensor.shape[0]
    num_rows = 1+ num_kernels // num_cols
    fig = plt.figure(figsize=(num_cols,num_rows))
    fig.set_facecolor("black")

    for i in range(tensor.shape[0]):
        ax1 = fig.add_subplot(num_rows,num_cols,i+1)
        ax1.imshow(tensor[i],)
        ax1.axis('off')
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
    
    fig.suptitle(superTitle, color='white')
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

In [ ]:
def getLabels(dataset):
    labels = []
    for i in range(len(dataset)):
        labels.append(dataset[i][1])
    return labels

def run_testing_labels(model, test_loader):

    # Test the model
    with torch.no_grad():
        correct = 0
        total = 0
        allPredLabels = []
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _ , predicted = torch.max(outputs, 1)
            allPredLabels.append(predicted)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        testAcc = 100 * correct / total
        print('Accuracy of the network on the {} test images: {} %'.format(total, testAcc))
        
        return testAcc, allPredLabels
    
def display_classification_results(y_test, y_pred,title="Accuracy Heatmap",show_report=True):
    print("Test accuracy: %.2f%%"%(sklearn.metrics.accuracy_score(y_test, y_pred)*100))
    if show_report:
        print("Classification Report:\n",sklearn.metrics.classification_report(y_test, y_pred))
    plt.rcParams['figure.figsize'] = (10.0, 8.0)
    conf_mat = sklearn.metrics.confusion_matrix(y_test,y_pred)
    print("Confusion Matrix:\n",conf_mat)
    ax = sns.heatmap(conf_mat);
    ax.set(title=title, xlabel="CIFAR10 Classes", ylabel="CIFAR10 Classes")